In [8]:
from http.client import HTTPSConnection
from urllib.parse import quote_plus, urlencode
import re
import json
import decimal
from datetime import date

class VehicleAPI:
    
    def __init__(self):
        self.host_opt = "ico.autotrader.ca"
        self.host_est = "www.autotrader.ca"
        self.regex = r'\$([0-9,]+) \- \$([0-9,]+)'
    
    def getAvailableYears(self):
        return [str(x) for x in range(date.today().year, 1999, -1)]
    
    def getAvailableMakes(self, year:str):
        conn = HTTPSConnection(self.host_opt)
        url = "/Ico/GetMakes?year="+quote_plus(year)
        conn.request("GET", url, headers={"Host": host_opt})
        return json.loads(conn.getresponse().read())["Makes"]
    
    def getAvailableModels(self, year:str, make:str):
        conn = HTTPSConnection(self.host_opt)
        url = "/Ico/GetModels?year="+quote_plus(year)+"&make="+quote_plus(make)
        conn.request("GET", url, headers={"Host": host_opt})
        return json.loads(conn.getresponse().read())["Models"]
        
    def getAvailableTrims(self, year:str, make:str, model:str):
        conn = HTTPSConnection(self.host_opt)
        url = "/Ico/GetTrims?year="+quote_plus(year)+"&make="+quote_plus(make)+"&model="+quote_plus(model)
        conn.request("GET", url, headers={"Host": host_opt})
        return json.loads(conn.getresponse().read())["Versions"]
    
    def getEstimatedValue(self, guid:str, mileage:str, postalCode:str, result_page="b7b49249-39ce-4a00-b6a2-3d8e07691944"):
        conn = HTTPSConnection(self.host_est)
        url = "/valuations/ResultPage/"+result_page
        params = urlencode({
            'guid': guid, 
            'mileage': mileage, 
            'postalCode': postalCode
        })
        conn.request("POST", url, params, headers={
            "Host": self.host_est, 
            "Content-type": "application/x-www-form-urlencoded",
            "Content-Length": len(params)
        })
        response = conn.getresponse()
        size = int(response.getheader('Content-Length', '0'))
        found = re.findall(self.regex, str(response.read(size)), re.MULTILINE)
        if len(found) == 1 and len(found[0]) == 2:
            minimum = decimal.Decimal(found[0][0].replace(',', ''))
            maximum = decimal.Decimal(found[0][1].replace(',', ''))
            return (minimum+maximum)/2
        return 0

print(VehicleAPI().getEstimatedValue("C430177", "20000", "L8S 1M9"))

31950


In [7]:
from http.client import HTTPSConnection
from urllib.parse import quote_plus, urlencode
import re
import json
import decimal

host_opt = "ico.autotrader.ca"
host_est = "www.autotrader.ca"
regex = r'\$([0-9,]+) \- \$([0-9,]+)'

# build options
conn_opt = HTTPSConnection(host_opt)

# choose year, get makes
opt_year = quote_plus("2023")
url_get_makes = "/Ico/GetMakes?year="+opt_year
print(url_get_makes)
conn_opt.request("GET", url_get_makes, headers={"Host": host_opt})
res = conn_opt.getresponse()
opt_makes = json.loads(res.read())
print(opt_makes)

# choose make, get models
opt_make = quote_plus(opt_makes["Makes"][0]["Make"])
url_get_models = "/Ico/GetModels?year="+opt_year+"&make="+opt_make
print(url_get_models)
conn_opt.request("GET", url_get_models, headers={"Host": host_opt})
res = conn_opt.getresponse()
opt_models = json.loads(res.read())
print(opt_models)

# choose model, get trims
opt_model = quote_plus(opt_models["Models"][0]["Model"])
url_get_trims = "/Ico/GetTrims?year="+opt_year+"&make="+opt_make+"&model="+opt_model
print(url_get_trims)
conn_opt.request("GET", url_get_trims, headers={"Host": host_opt})
res = conn_opt.getresponse()
opt_trims = json.loads(res.read())
print(opt_trims)

# choose trim, validate trim (optional)
opt_trim_guid = quote_plus(opt_trims["Versions"][0]["Guid"])
opt_trim_source = str(opt_trims["Versions"][0]["Source"])
url_validate_trim = "/Ico/validateTrim?guid="+opt_trim_guid+"&source="+opt_trim_source
print(url_validate_trim)
conn_opt.request("GET", url_validate_trim, headers={"Host": host_opt})
res = conn_opt.getresponse()
opt_validate_trim = json.loads(res.read())
print(opt_validate_trim)

# Get estimated value
conn_est = HTTPSConnection(host_est)
mileage = 20000
postalCode = "L8S 1M9"
url_est = "/valuations/ResultPage/b7b49249-39ce-4a00-b6a2-3d8e07691944"
print(url_est)
params_est = urlencode({
    'guid': opt_trim_guid, 
    'mileage': mileage, 
    'postalCode': postalCode
})
print(params_est)
print(len(params_est))
conn_est.request("POST", url_est, params_est, headers={
    "Host": host_est, 
    "Content-type": "application/x-www-form-urlencoded",
    "Content-Length": len(params_est)
})
res = conn_est.getresponse()
length = int(res.getheader('Content-Length', '0'))
raw = str(res.read(length))

matches = re.findall(regex, raw, re.MULTILINE)
print(matches)
if len(matches) == 1 and len(matches[0]) == 2:
    min = decimal.Decimal(matches[0][0].replace(',', ''))
    max = decimal.Decimal(matches[0][1].replace(',', ''))
    print(matches[0][0], "+", matches[0][1], "/ 2 =", (min+max)/2)
else:
    print("Params [guid =", opt_trim_guid, " mileage =", mileage, " postalCode =", postalCode, "] invalid or can't be estimated")

/Ico/GetMakes?year=2023
{'Makes': [{'Make': 'ACURA'}, {'Make': 'ALFA ROMEO'}, {'Make': 'AUDI'}, {'Make': 'BMW'}, {'Make': 'BUICK'}, {'Make': 'CADILLAC'}, {'Make': 'CHEVROLET'}, {'Make': 'CHRYSLER'}, {'Make': 'DODGE'}, {'Make': 'FIAT'}, {'Make': 'FORD'}, {'Make': 'GENESIS'}, {'Make': 'GMC'}, {'Make': 'HONDA'}, {'Make': 'HYUNDAI'}, {'Make': 'INFINITI'}, {'Make': 'JAGUAR'}, {'Make': 'JEEP'}, {'Make': 'KIA'}, {'Make': 'LAND ROVER'}, {'Make': 'LEXUS'}, {'Make': 'LINCOLN'}, {'Make': 'MASERATI'}, {'Make': 'MAZDA'}, {'Make': 'MERCEDES-BENZ'}, {'Make': 'MINI'}, {'Make': 'MITSUBISHI'}, {'Make': 'NISSAN'}, {'Make': 'POLESTAR'}, {'Make': 'PORSCHE'}, {'Make': 'RAM'}, {'Make': 'SUBARU'}, {'Make': 'TESLA'}, {'Make': 'TOYOTA'}, {'Make': 'VOLKSWAGEN'}, {'Make': 'VOLVO'}, {'Make': 'WAGONEER'}]}
/Ico/GetModels?year=2023&make=ACURA
{'Models': [{'Model': 'INTEGRA'}, {'Model': 'MDX'}, {'Model': 'RDX'}, {'Model': 'TLX'}]}
/Ico/GetTrims?year=2023&make=ACURA&model=INTEGRA
{'Versions': [{'Trim': 'A-SPEC CVT', '

### API by Acu-Trade

#### Estimate by car details

##### Get avail makes
Pass in "Year"
GET `https://ico.autotrader.ca/Ico/GetMakes?year=2023`
Response
```json
{
  "Makes": [
    { "Make": "ACURA" },
    { "Make": "ALFA ROMEO" },
    { "Make": "AUDI" },
    { "Make": "BMW" },
    { "Make": "BUICK" },
    { "Make": "CADILLAC" },
    { "Make": "CHEVROLET" },
    { "Make": "CHRYSLER" },
    { "Make": "DODGE" },
    { "Make": "FORD" },
    { "Make": "GENESIS" },
    { "Make": "GMC" },
    { "Make": "HONDA" }, // pass into next...
    { "Make": "HYUNDAI" },
    { "Make": "INFINITI" },
    { "Make": "JAGUAR" },
    { "Make": "JEEP" },
    { "Make": "KIA" },
    { "Make": "LAND ROVER" },
    { "Make": "LEXUS" },
    { "Make": "LINCOLN" },
    { "Make": "MASERATI" },
    { "Make": "MAZDA" },
    { "Make": "MERCEDES-BENZ" },
    { "Make": "MINI" },
    { "Make": "MITSUBISHI" },
    { "Make": "NISSAN" },
    { "Make": "PORSCHE" },
    { "Make": "RAM" },
    { "Make": "ROLLS-ROYCE" },
    { "Make": "SUBARU" },
    { "Make": "TOYOTA" },
    { "Make": "VOLKSWAGEN" },
    { "Make": "VOLVO" },
    { "Make": "WAGONEER" }
  ]
}
```

##### Get avail models
Pass in "Year", "Make"
GET `https://ico.autotrader.ca/Ico/GetModels?year=2023&make=HONDA`
Response
```json
{
  "Models": [
    { "Model": "ACCORD HYBRID" }, // pass into next...
    { "Model": "ACCORD SEDAN" },
    { "Model": "CIVIC HATCHBACK" },
    { "Model": "CIVIC SEDAN" },
    { "Model": "CIVIC SI SEDAN" },
    { "Model": "CR-V" },
    { "Model": "CR-V HYBRID" },
    { "Model": "HR-V" },
    { "Model": "ODYSSEY" },
    { "Model": "PASSPORT" },
    { "Model": "PILOT" },
    { "Model": "RIDGELINE" }
  ]
}
```

##### Get avail trims
Pass in "Year", "Make", "Model"
GET `https://ico.autotrader.ca/Ico/GetTrims?year=2023&make=HONDA&model=ACCORD%20HYBRID`
Response
```json
{
  "Versions": [
    {
      "Trim": "SPORT ECVT",
      "Guid": "C438174", // pass into next...
      "Specialized": false,
      "Source": 1 // pass into next...
    },
    {
      "Trim": "TOURING ECVT",
      "Guid": "C448275",
      "Specialized": false,
      "Source": 1
    }
  ]
}
```

##### Validate trim
Pass in "Guid", "Source"
GET `https://ico.autotrader.ca/Ico/validateTrim?guid=C438174&source=1`
Response
```json
{ "IsValid": true }
```

##### Get estimated value from result page
Clear cookie before use
POST `https://www.autotrader.ca/valuations/ResultPage/6a1b1c2e-b1c3-468e-be7e-8afe393b4be9`
Header
```
Host: www.autotrader.ca
Content-Type: application/x-www-form-urlencoded
Content-Length: 45
```
Body `guid=C438174&mileage=20000&postalCode=L8B+1A1`
Use Regex `\$([0-9,]+) \- \$([0-9,]+)` (1-min, 2-max) to find value range in response text.
